### Import Statements and Initializing our tagger and dataframes

In [2]:
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from nltk.corpus import names
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import nltk, string
from nltk.corpus import brown, stopwords
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
punctuations = list(string.punctuation)
from sklearn.cross_validation import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, f_classif
lemmatizer = WordNetLemmatizer()
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import cross_val_score
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [6]:
def create_data_sets(sentences):
    size = int(len(sentences) * 0.9)
    train_sents = sentences[:size]
    test_sents = sentences[size:]
    return train_sents, test_sents

def build_backoff_tagger (train_sents):
    t0 = nltk.DefaultTagger('NN')
    t1 = nltk.UnigramTagger(train_sents, backoff=t0)
    t2 = nltk.BigramTagger(train_sents, backoff=t1)
    t3 = nltk.TrigramTagger(train_sents, backoff=t2)
    return t3


def train_tagger(already_tagged_sents):
    train_sents, test_sents = create_data_sets(already_tagged_sents)
    ngram_tagger = build_backoff_tagger(train_sents)
    print ("%0.3f pos accuracy on test set" % ngram_tagger.evaluate(test_sents))
    return ngram_tagger

In [7]:
def train_tagger_on_brown():
    modified_speech_sents = [[('common', 'JJ'), ('Hard-working', 'JJ'), ('people', 'NNS'), ('.', '.')],
                        [("I'm", 'PPSS+BEM'), ('a', 'AT'), ('Republican', 'NP'), ('.', '.')],
                        [("I'm", 'PPSS+BEM'), ('Republican', 'NP'),('.', '.')], 
                        [('the', 'AT'), ('Republican', 'NP'), ('politicians', 'NNS'), ('.', '.')],
                        [('the', 'AT'), ('American', 'NP'), ('people', 'NNS'), ('.', '.')]]


    brown_tagged_sents = brown.tagged_sents(categories=['adventure', 'editorial', 'fiction', 'government', 'hobbies',
    'humor', 'learned', 'lore', 'mystery', 'religion', 'reviews', 'romance'])
    
    all_tagged_sents = modified_speech_sents + brown_tagged_sents
    return train_tagger(all_tagged_sents)

In [8]:
def Trained_Speech_Tagger(sents,tagger):
    if(isinstance(sents[0],list)):
        return [tagger.tag(sent) for sent in sents]
    else:
        return tagger.tag(sents)

In [9]:
brown_tagger = train_tagger_on_brown()

0.909 pos accuracy on test set


In [10]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

In [3]:
df = pd.read_csv('tweet_training.csv')

In [4]:
dt = pd.read_csv('total_tweets.csv')

### Finding unique hashtags in the tweets

In [5]:
hashtags = dt.hash_tags.unique()
hashtags

array(['[]', "['#debates', '#debates']", "['#BasementDwellers']", ...,
       "['#markets', '#election', '#trump']",
       "['#Obama', '#failed', '#maga', '#trump']", "['#Michigan']"], dtype=object)

In [11]:
new_hash = []
for item in hashtags:
    item = item.replace('[]', '')
    item = item.replace('[', '')
    item = item.replace(']', '')
    item = item.replace('\'','')
    item = item.replace(',','')
    item = item.lower()
    new_item = item.split(" ")
    for x in new_item:
        #print(x)
        new_hash.append(x)

In [12]:
print(new_hash)

['', '#debates', '#debates', '#basementdwellers', '#bigleaguetruth', '#trump', '#vpdebate', '#vpdebate', '#timkaine', '#imwithher', '#vpdebate', '#de', '#privateequity', '#tuesdaymotivation', '#vpdebate', '#hypocrite', '#vpdebate', '#vpdebate', '#mikepence', '#teamiphone', '#debates', '#', '#guccifer2', '#clintonfoundation', '#honduras', '#vpdebate', '#imwithher', '#trumppence16', '#americ', '#guccifer2', '#tcot', '#wakeupamerica', '#maga', '#vpdeba', '#assange', '#vpde', '#crookedhillar', '#stonetear', '#potus', '#vpdebate', '#guiltyassin', '#vpde', '#clinton', '#vpdebate', '#decision2016', '#obamacare', '#cato2016', '#debat', '#debates', '#debate2016', '#vpdebate', '#trumppence16', '#amer', '#conflictofinterest', '#maga', '#vpdebate', '#vpdebate2016', '#debates', '#vpdebate', '#clinton', '#vpdebate', '#election2016', '#vicepresidentialdebate', '#vpdebate', '#bigleaguetruth', '#vpdeb', '#vpdebate', '#election2016', '#mhmtdeb', '#vpdebate', '#guccifer20', '#kaine', '#clinton', '#vpdeba

In [15]:
from nltk.probability import FreqDist
fdist1 = FreqDist(new_hash)
fdist1.most_common(20)

[('#vpdebate', 352),
 ('#trump', 265),
 ('#hillary', 113),
 ('#maga', 79),
 ('#clinton', 69),
 ('#pence', 63),
 ('#kaine', 56),
 ('#imwithher', 42),
 ('#', 41),
 ('#neverhillary', 34),
 ('#tcot', 32),
 ('#trumppence16', 31),
 ('#nevertrump', 29),
 ('#debates', 26),
 ('#timkaine', 26),
 ('#mikepence', 26),
 ('#trump2016', 25),
 ('#crookedhillary', 24),
 ('#debates2016', 23),
 ('#trumptrain', 23)]

In [16]:
set1 = set(new_hash)
result = list(set1)
len(result)

1237

In [17]:
tweet_count = []
for idx, item in enumerate(dt['hash_tags']):
    for x in ["hillary", "clinton", "tim", "kaine", "pence", "donald", "trump", "america", "her", "maga"]:
        if x in item.lower():
            tweet_count.append(idx)

In [18]:
set2 = set(tweet_count)
result1 = list(set2)
len(result1)

4025

In [19]:
dt['text'][8272]

"RT @JohnFromCranber: REPORT: All 3 Clinton Called The Secret Service 'Pigs' https://t.co/PVR9rSwlc9 Appears #Hillary's The One With a 'Tempâ€¦"

### Part of speech tagging for the tweets to understand usage of nouns, adjectives and adverbs

In [20]:
tw = pd.read_csv('tweets_harman.csv')

In [21]:
filt = tw["Winner"].notnull()

In [22]:
filt_tw = tw[filt]

In [23]:
hilltw = filt_tw[filt_tw.Winner == 0.0]

In [24]:
trumptw = filt_tw[filt_tw.Winner == 1.0]

In [25]:
len(trumptw['Tweet'])

232

In [26]:
len(hilltw)

485

#### Tagging tweets with our backoff tagger

In [27]:
tagged_tweet_hillary = []
for item in hilltw['Tweet']:
    tagged_tweet_hillary.append(Trained_Speech_Tagger(tknzr.tokenize(item),brown_tagger))
    
tagged_tweet_trump = []
for item in trumptw['Tweet']:
    tagged_tweet_trump.append(Trained_Speech_Tagger(tknzr.tokenize(item),brown_tagger))

#### Compute the noun, adjective and verb count for tweets in the Hillary camp

In [28]:
hillary_nouns = 0
for item in tagged_tweet_hillary:
#     print ("new tweet")
#     count = 0
    for value in item:
        if value[1].find("NN") != -1:
            #print(value)
            #count = count + 1
            hillary_nouns = hillary_nouns + 1
    #print(count)
    #hilnli.append(count)

print(hillary_nouns)
print(len(tagged_tweet_hillary))
print(hillary_nouns/float(len(tagged_tweet_hillary)))
#print(hilnli)

3072
485
6.334020618556701


In [29]:
hillary_adj = 0
for item in tagged_tweet_hillary:
#     print ("new tweet")
#     count = 0
    for value in item:
        if value[1].find("JJ") != -1:
            #print(value)
            #count = count + 1
            hillary_adj = hillary_adj + 1
    #print(count)
    #hilnli.append(count)

print(hillary_adj)
print(len(tagged_tweet_hillary))
print(hillary_adj/float(len(tagged_tweet_hillary)))
#print(hilnli)

337
485
0.6948453608247422


In [30]:
hillary_vb = 0
for item in tagged_tweet_hillary:
#     print ("new tweet")
#     count = 0
    for value in item:
        if value[1].find("VB") != -1:
            #print(value)
            #count = count + 1
            hillary_vb = hillary_vb + 1
    #print(count)
    #hilnli.append(count)

print(hillary_vb)
print(len(tagged_tweet_hillary))
print(hillary_vb/float(len(tagged_tweet_hillary)))
#print(hilnli)

1046
485
2.156701030927835


#### Compute the noun, adjective and verb count for tweets in the Trump camp

In [31]:
trump_vb = 0
for item in tagged_tweet_trump:
#     print ("new tweet")
#     count = 0
    for value in item:
        if value[1].find("VB") != -1:
            #print(value)
            #count = count + 1
            trump_vb = trump_vb + 1
    #print(count)
    #hilnli.append(count)

print(trump_vb)
print(len(tagged_tweet_trump))
print(trump_vb/float(len(tagged_tweet_trump)))

421
232
1.8146551724137931


In [32]:
trump_aj = 0
for item in tagged_tweet_trump:
#     print ("new tweet")
#     count = 0
    for value in item:
        if value[1].find("JJ") != -1:
            #print(value)
            #count = count + 1
            trump_aj = trump_aj + 1
    #print(count)
    #hilnli.append(count)

print(trump_aj)
print(len(tagged_tweet_trump))
print(trump_aj/float(len(tagged_tweet_trump)))

172
232
0.7413793103448276


In [33]:
trump_nn = 0
for item in tagged_tweet_trump:
#     print ("new tweet")
#     count = 0
    for value in item:
        if value[1].find("NN") != -1:
            #print(value)
            #count = count + 1
            trump_nn = trump_nn + 1
    #print(count)
    #hilnli.append(count)

print(trump_nn)
print(len(tagged_tweet_trump))
print(trump_nn/float(len(tagged_tweet_trump)))

1383
232
5.961206896551724


### A deep dive into the adjective list for Trump and Hillary

In [34]:
trump_adj_list = []
for item in tagged_tweet_trump:
#     print ("new tweet")
#     count = 0
    for value in item:
        if value[1].find("JJ") != -1:
            trump_adj_list.append(value[0])
            #count = count + 1
            
    #print(count)
    #hilnli.append(count)

#print(trump_vb_list)
fdistta = FreqDist(trump_adj_list)
fdistta.most_common(25)

[('American', 6),
 ('great', 5),
 ('wrong', 4),
 ('foreign', 4),
 ('good', 4),
 ('political', 4),
 ('such', 4),
 ('right', 4),
 ('deplorable', 4),
 ('own', 3),
 ('open', 3),
 ('better', 3),
 ('rude', 3),
 ('worse', 3),
 ('economic', 3),
 ('smart', 2),
 ('Muslim', 2),
 ('big', 2),
 ('entire', 2),
 ('inner', 2),
 ('Full', 2),
 ('hard', 2),
 ('real', 2),
 ('presidential', 2),
 ('Nuclear', 2)]

In [35]:
hillary_adj_list = []
for item in tagged_tweet_hillary:
#     print ("new tweet")
#     count = 0
    for value in item:
        if value[1].find("JJ") != -1:
            hillary_adj_list.append(value[0])
            #count = count + 1
            
    #print(count)
    #hilnli.append(count)

#print(hillary_vb_list)
fdistha = FreqDist(hillary_adj_list)
fdistha.most_common(25)

[('nuclear', 11),
 ('own', 10),
 ('bad', 9),
 ('good', 9),
 ('great', 7),
 ('wrong', 6),
 ('presidential', 6),
 ('false', 6),
 ('new', 6),
 ('public', 4),
 ('deplorable', 4),
 ('strong', 4),
 ('scary', 4),
 ('military', 3),
 ('New', 3),
 ('old', 3),
 ('Atlantic', 3),
 ('important', 3),
 ('political', 3),
 ('Latest', 3),
 ('economic', 3),
 ('crazy', 3),
 ('worst', 3),
 ('best', 3),
 ('smart', 3)]

In [36]:
print(fdistha.most_common(35))
print ()
print(fdistta.most_common(35))

[('nuclear', 11), ('own', 10), ('bad', 9), ('good', 9), ('great', 7), ('wrong', 6), ('presidential', 6), ('false', 6), ('new', 6), ('public', 4), ('deplorable', 4), ('strong', 4), ('scary', 4), ('military', 3), ('New', 3), ('old', 3), ('Atlantic', 3), ('important', 3), ('political', 3), ('Latest', 3), ('economic', 3), ('crazy', 3), ('worst', 3), ('best', 3), ('smart', 3), ('profound', 3), ('open', 3), ('rich', 3), ('true', 3), ('perfect', 2), ('flat', 2), ('negative', 2), ('catastrophic', 2), ('criminal', 2), ('successful', 2)]

[('American', 6), ('great', 5), ('wrong', 4), ('foreign', 4), ('good', 4), ('political', 4), ('such', 4), ('right', 4), ('deplorable', 4), ('own', 3), ('open', 3), ('better', 3), ('rude', 3), ('worse', 3), ('economic', 3), ('smart', 2), ('Muslim', 2), ('big', 2), ('entire', 2), ('inner', 2), ('Full', 2), ('hard', 2), ('real', 2), ('presidential', 2), ('Nuclear', 2), ('crooked', 2), ('true', 2), ('strong', 2), ('funny', 2), ('fair', 2), ('little', 2), ('nuclear'

In [37]:
xt_h = set([word.lower() for word in trump_adj_list]) - set([word.lower() for word in hillary_adj_list])
xh_t = set([word.lower() for word in hillary_adj_list]) - set([word.lower() for word in trump_adj_list])

### Creating a unique set of adjectives for each camp to create our bag of words

In [38]:
print(xt_h)
print()
print(xh_t)

{'red', 'obvious', 'short', 'complete', 'intentional', 'full', 'blatant', 'magical', 'crooked', 'stronger', 'secret', 'bold', 'corrupt', 'valid', 'super', 'annoying', 'impeccable', 'friendly', 'free', 'due', 'fine', 'little', 'syrian', 'sick', 'entire', 'level', 'teenage', 'personal', 'coherent', 'downcast', 'safe', 'intelligent', 'rude', 'severe', 'challenging', 'disastrous', 'gay', 'comfortable', 'inner', 'shorter', 'arrogant', 'african', 'previous', 'democratic', 'worse', 'wont'}

{'outrageous', 'tight', 'tough', 'effective', 'willing', 'entertaining', 'selfish', 'hardworking', 'live', 'nasty', 'responsible', 'chief', 'negative', 'clean', 'respectful', 'hot', 'unlikely', 'direct', 'giant', 'mean', 'accurate', 'insane', 'local', 'catastrophic', 'safer', 'criminal', 'financial', 'pure', 'successful', 'baffling', 'terrible', 'highest', 'horrible', 'predictable', 'small', 'fat', 'current', 'foolish', 'closer', 'international', 'ignorant', 'mad', 'harsh', 'ready', 'like', 'popular', 'lat